In [1]:
from classiq import * 
import math 

In [ ]:
# TODO Initialize ancilla registers with the correct quantum states
# NOTE Our b vector is zero, therefore C-V_S2 and C-U_b do not need to be implemented, but could be for future purposes

K = 10 # Accuracy of solution - subject to change
T = math.ceil(math.log2(K+1))
t = 0 # Time of evaluation of output

print("Register sizes:", 1, T, 1)

# TODO Create the gate C-V_S1 and apply it to prepare the ancilla qubits
@qfunc
def prepare_ancilla_regs(ancilla_reg_1: QBit, ancilla_reg_2: QBit):
   Z(ancilla_reg_1)
   
   pass 


   

# TODO Create the gate C-Ux and apply it to encode |x0> into the work register
@qfunc
def encode_x0(ancilla_reg_1: QBit, work_reg: QBit):
   pass

@qfunc
def apply_ui(iteration: CInt, ancilla_reg_2: QNum, work_reg: QBit):
   control(ctrl=(ancilla_reg_2 == iteration), stmt_block=lambda: Y(work_reg))

@qfunc
def decode():
   pass

@qfunc
def main(ancilla_reg_1: Output[QBit], ancilla_reg_2: Output[QNum], work_reg: Output[QBit]):
   allocate(1, ancilla_reg_1)
   allocate(T, ancilla_reg_2)
   allocate(1, work_reg) # 2x2 matrix, will change for higher order ODEs if we get to it
   
   # prepare_ancilla_regs(ancilla_reg_1, ancilla_reg_2)
   # encode_x0(ancilla_reg_1, work_reg)

   repeat(count=K+1, iteration=lambda i: if_(condition=(i % 2 != 0), then=lambda: apply_ui(i, ancilla_reg_2, work_reg)))


In [ ]:
# View circuit on Classiq IDE
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

### Encoding


In [3]:
# #Log base 2 (N) work qubits needed to encode the N-dimensional vectors |x(0)>

# Need Log_2 N work qubits. 

# vector x and vector b are preped and stored by the work qubits 





k = 7 
T = math.ceil(math.log(k+1, 2))

@qfunc 



SyntaxError: invalid syntax (331670662.py, line 3)